In [207]:
import os
import math

In [208]:
class Point:
    def __init__(self, x, y, z):
        self.x = x
        self.y = y
        self.z = z

    def __call__(self) -> tuple:
        return (self.x, self.y, self.z)

    def __str__(self) -> str:
        return str((self.x, self.y, self.z))

    def __eq__(self, other):
        return self.__dict__ == other.__dict__


class Edge:
    def __init__(self, p1, p2):
        self.p1 = p1
        self.p2 = p2


class Face:
    def __init__(self, p1, p2, p3):
        self.p1 = p1
        self.p2 = p2
        self.p3 = p3

        self.e1 = Edge(p1, p2)
        self.e2 = Edge(p2, p3)
        self.e3 = Edge(p3, p1)
        
    def __str__(self):
        return "{} {} {}".format(str(self.p1), str(self.p2), str(self.p3))

In [209]:
def cross(a, b):
    return Point(a.y*b.z-a.z*b.y, a.z*b.x-a.x*b.z, a.x*b.y-a.y*b.x)

def dot(a, b):
    return (a.x*b.x)+(a.y*b.y)+(a.z*b.z)

def norma(a):
    return math.sqrt(a.x*a.x + a.y*a.y + a.z*a.z)

def angle(a, b):
    return dot(a, b) / norma(a)*norma(b)

def sub(a, b):
    return Point(a.x-b.x, a.y-b.y, a.z-b.z)

def normal(face):
    v1 = sub(face.p2, face.p1)    
    v2 = sub(face.p3, face.p1)
    return cross(v1,v2)

In [210]:
def find_min(Points):
    ys = [p.y for p in Points]
    min_index = ys.index(min(ys))
    return Points[min_index]

In [211]:
def adjacente(P, F, e):
    ang_min = 99999999
    candidate = None
    normal1 = normal(F)
#     print(normal1)
    for p in P:
#         print("p", p)
        if p == F.p1 or p == F.p2 or p == F.p3: continue
        normal2 = normal(Face(e.p2, e.p1, p))
        ang = -1*angle(normal1, normal2)
#         print('n2',normal2)
#         print("ANGLE", ang)
        if ang < ang_min:
            ang_min = ang
            candidate = p
    return candidate

In [212]:
def find_first_face(P):
    p1 = find_min(P)
    p2 = Point(p1.x+1, p1.y, p1.z)    
    p3 = Point(p1.x, p1.y, p1.z+1)
    
    ftemp = Face(p1,p2,p3)
    
    p3 = adjacente(P, ftemp, ftemp.e1)
    
    ftemp2 = Face(p1,p2,p3)
    p2 = adjacente(P, ftemp2, ftemp2.e3)
    
    return Face(p1,p2,p3)    

In [213]:
a = adjacente([Point(0,1,0), Point(1,0,0), Point(0,0,1), Point(0,-1,0), Point(0,0,0), Point(0, 0,0.5), Point(0, 0, 4), Point(0, -5,0)], Face(Point(0,1,0), Point(1,0,0), Point(0,0,1)), Edge(Point(0,1,0), Point(1,0,0)))

In [214]:
def gift_wrap(P):
    f = find_first_face(P)
    Q = [f]
    hull = [f]
    while len(Q) > 0:
        face = Q.pop(0)
        arestas = [face.e1, face.e2, face.e3]
        for aresta in arestas:
            p_ = adjacente(P, face, aresta)
            f_ = Face(aresta.p2, aresta.p1, p_)
            hull.append(f_)
    return hull

In [215]:
def read_vertices_from_obj(objPath):
    points = []
    with open(objPath, 'r') as f:
        for line in f:
            line = line.replace('v', '')
            line_ = line.split()
            try:
                points.append(Point(float(line_[0]), float(line_[1]), float(line_[2])))
            except:
                return points
    return points

In [228]:
def write_faces(F, P, filename=False):
    i1 = 0
    i2 = 0
    i3 = 0
    if not filename:
        filename = os.path.join(os.getcwd(), 'obj', 'faces.obj')
    with open(filename, 'w') as f:
        for point in P:
            f.write("v {} {} {}".format(point.x, point.y, point.z))
            f.write('\n')
        f.write('\n')
        for i in range(len(F)):
            for j in range(len(P)):
                if F[i].p1.x == P[j].x and F[i].p1.y == P[j].y and F[i].p1.z == P[j].z:
                    i1 = j
                if F[i].p2.x == P[j].x and F[i].p2.y == P[j].y and F[i].p2.z == P[j].z:
                    i2 = j
                if F[i].p3.x == P[j].x and F[i].p3.y == P[j].y and F[i].p3.z == P[j].z:
                    i3 = j
            f.writelines("f {} {} {}".format(i1+1, i2+1, i3+1))
            f.write('\n')


In [229]:
pth = "/Users/romul/Documents/UFC/MESTRADO/Disciplinas/Geometria Computacional/obj/"

In [230]:
cilindro = read_vertices_from_obj(pth+"Debug.obj")
# for c in cilindro: print(c)

In [231]:
ch = gift_wrap(cilindro)
write_faces(ch, cilindro)